# Chapter 7.1.0 - Properties

In [3]:
class Person:
    def __init__(self,firstName,lastName):
        self.firstName = firstName
        self.lastName = lastName

    @property # property decorator - this function is only a readable property now
    def fullName(self):
        return f"{self.firstName} {self.lastName}"

johnWalter = Person('John','Walter')
johnWalter.fullName

'John Walter'

In [6]:
class Person:
    def __init__(self,firstName,lastName):
        self.firstName = firstName
        self.lastName = lastName

    @property 
    def fullName(self):
        return f"{self.firstName} {self.lastName}"

    @fullName.setter
    def fullName(self,value):
       self.firstName,self.lastName = value.split(' ',1)

kandy = Person('Kandy','Woods')
kandy.fullName 
#After marriage
kandy.fullName = 'Kandy Walker'
kandy.fullName

'Kandy Walker'

In [9]:
# back public readable variables by no-readable ones
class Amount:
    def __init__(self,amount):
        self._amount = amount # javascript also add an underscore to "private properties"    
    @property
    def amount(self):
        return self._amount 

cost = Amount(1.23)
display(cost.amount)
cost.amount = 1.50

1.23

AttributeError: property 'amount' of 'Amount' object has no setter

In [12]:
class Ticket:
    def __init__(self,price):
        self.price = price     
    @property
    def price(self):
        return self._price
    @price.setter
    def price(self,value):
        if value < 0:
            raise ValueError('Not in my house')
        self._price = value 

tygaTickets = Ticket(237)
tygaTickets.price = 200
tygaTickets.price


200

In [13]:
tygaTickets.price = -50

ValueError: Not in my house

# Section 7.1.1 - Properties and Refactoring

In [1]:
# Imagine Before
# class Money:
#     def __init__(self,dollars,cents):
#         self.dollars = dollars
#         self.cents = cents 

# And After
class Money:
    def __init__(self,dollars,cents):
        self.total_cents = dollars * 100 + cents 

In [2]:
#code written like this during the before will break
myWallet = Money(500,32)
myWallet.price

AttributeError: 'Money' object has no attribute 'price'

In [21]:
# To fix, properties are used
class Money:
    def __init__(self,dollars,cents):
        self.total_cents = dollars * 100 + cents 

    @property
    def dollars(self):
        return self.total_cents / 100)
    
    @dollars.setter
    def dollars(self,newDollars):
        self.total_cents = 100 * newDollars + self.cents

    @property
    def cents(self):
        return self.total_cents % 100 
    
    @cents.setter
    def dollars(self,newCents):
        self.total_cents = newCents + self.dollars * 100

myMoney = Money(50,0)

    

SyntaxError: unmatched ')' (147155631.py, line 8)

In [22]:
display(myMoney.cents,
myMoney.dollars,
myMoney.total_cents
)

50

50

50

In [20]:
myMoney.dollars

50

# Section 7.2 The Factory Patterns

### Section 7.2.1 Alternative Constructors: The Simple Factory

In [32]:
# can use class methods to handle creating the same object in different variations

class Money:
    def __init__(self,dollars,cents):
        self.dollars = dollars
        self.cents = cents 
    @classmethod
    def moneyFromPennies(cls,pennies):
        dollars = pennies // 100
        cents = pennies % 100
        return cls(dollars,cents)
    @classmethod
    def moneyFromQuarters(cls,quarters):
        dollars = quarters // 4
        cents = quarters % 4 * 25
        return cls(dollars,cents)

regularMoney = Money(5,25)
pennies = Money.moneyFromPennies(525)
quarters = Money.moneyFromQuarters(21)
print(f"Regular money in dollars is {regularMoney.dollars} and {regularMoney.cents} cents")
print(f"pennies in dollars is {pennies.dollars} and {pennies.cents} cents")
print(f"quarters in dollars is {quarters.dollars} and {quarters.cents} cents")

Regular money in dollars is 5 and 25 cents
pennies in dollars is 5 and 25 cents
quarters in dollars is 5 and 25 cents


# Section 7.2.2 Dynamic Type: The Factory Method Pattern